In [1]:
import pandas as pd


df = pd.read_csv('data/results_21.csv',encoding='iso-8859-8') # for 21 to 24  elections

# df = pd.read_csv('data/results_25.csv') # for 25 election

df.head()

,שם ישוב,סמל ישוב,מספר קלפי,בזב,מצביעים,פסולים,כשרים,אמת,ג,דעם,...,צק,ץ,ץז,ץי,ק,קי,קן,קף,ר,שס
0,אבו ג'ווייעד (שבט),967,1.0,546,64,2,62,2,0,31,...,0,0,0,0,0,0,1,0,0,0
1,אבו ג'ווייעד (שבט),967,2.0,739,113,2,111,2,0,69,...,0,0,0,0,0,2,0,0,0,0
2,אבו ג'ווייעד (שבט),967,3.0,630,98,1,97,1,0,75,...,0,0,0,0,0,0,0,0,0,0
3,אבו גוש,472,1.0,518,197,6,191,4,0,36,...,0,1,0,0,1,0,0,0,0,1
4,אבו גוש,472,2.0,679,290,3,287,7,0,69,...,0,0,0,0,0,0,0,1,0,2


In [5]:


df = pd.read_excel('data/results_17.xls') # for 14 to 19  elections
df.head()

,מספר קלפי,שם ישוב,כתובת,מצביעים,כשרים,אמת,ג,ד,ה,ו,...,פכ,פץ,ף,צה,ק,קז,קנ,קפ,רק,שס
0,1010,מעטפות כפולות,NaN,1195,1175,183,23,12,3,7,...,0,2,1,0,0,0,58,0,38,90
1,1020,מעטפות כפולות,NaN,1210,1192,159,15,14,1,8,...,0,3,0,0,0,1,69,1,38,94
2,1030,מעטפות כפולות,NaN,1169,1148,153,15,2,0,5,...,0,0,0,1,1,2,51,0,39,87
3,1040,מעטפות כפולות,NaN,1209,1190,174,13,1,0,4,...,1,1,0,1,0,0,64,0,46,65
4,1050,מעטפות כפולות,NaN,1198,1176,163,12,8,1,9,...,0,1,0,1,1,0,55,0,44,92
